# Modélisation V2 - Avec Toutes les Nouvelles Features

## Objectif

Comparer les performances du modèle **avant** et **après** l'ajout des nouvelles features.

## Baseline (avant)

- **R² max : ~0.41** avec Random Forest
- Features : Landsat + TerraClimate (~35 features)

## Nouvelles features ajoutées

| Source | Features | Description |
|--------|----------|-------------|
| Landsat V2 | +10 | Stats (std) + buffer 200m |
| TerraClimate V2 | +24 | Lags, cumuls, anomalies |
| ESA WorldCover | +8 | % occupation du sol |
| SoilGrids | +6 | Propriétés du sol |
| DEM | +3 | Altitude, pente, orientation |
| Water Type | +1 | Rivière / lac |

## Modèles à tester

1. Random Forest (baseline)
2. XGBoost
3. LightGBM

In [ ]:
# Installation des dépendances si nécessaire
!pip install xgboost lightgbm --quiet

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Modèles
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb
import lightgbm as lgb

print("Imports OK!")

---

## Étape 1 : Charger les données

In [ ]:
# Charger les données fusionnées
train_df = pd.read_csv("../data/processed/merged_training.csv")
test_df = pd.read_csv("../data/processed/merged_validation.csv")

print(f"Training : {train_df.shape}")
print(f"Validation : {test_df.shape}")

print(f"\nColonnes : {len(train_df.columns)}")

In [ ]:
# Variables cibles (noms réels dans le dataset)
TARGET_COLS = ['Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']

# Noms courts pour l'affichage
TARGET_NAMES = ['Alkalinity', 'Conductivity', 'Phosphorus']

# Colonnes à exclure des features
EXCLUDE_COLS = ['Latitude', 'Longitude', 'Sample Date'] + TARGET_COLS

# Features
feature_cols = [c for c in train_df.columns if c not in EXCLUDE_COLS]
print(f"Nombre de features : {len(feature_cols)}")
print(f"Variables cibles : {TARGET_COLS}")

---

## Étape 2 : Préparation des données

In [ ]:
# =============================================================================
# PRÉPARATION DES FEATURES
# =============================================================================

def prepare_features(df, feature_cols):
    """
    Prépare les features pour la modélisation.
    - Encode les variables catégorielles
    - Gère les valeurs manquantes
    """
    X = df[feature_cols].copy()
    
    # Encoder water_type si présent
    if 'water_type' in X.columns:
        le = LabelEncoder()
        X['water_type'] = le.fit_transform(X['water_type'].fillna('unknown'))
    
    # Convertir toutes les colonnes en numérique
    for col in X.columns:
        if X[col].dtype == 'object':
            try:
                X[col] = pd.to_numeric(X[col], errors='coerce')
            except:
                X[col] = LabelEncoder().fit_transform(X[col].fillna('unknown'))
    
    # Remplir les valeurs manquantes par la médiane
    X = X.fillna(X.median())
    
    return X

# Préparer les features
X_train = prepare_features(train_df, feature_cols)
X_test = prepare_features(test_df, feature_cols)

print(f"X_train : {X_train.shape}")
print(f"X_test : {X_test.shape}")

# Vérifier les valeurs manquantes
print(f"\nValeurs manquantes (train) : {X_train.isnull().sum().sum()}")
print(f"Valeurs manquantes (test) : {X_test.isnull().sum().sum()}")

In [ ]:
# =============================================================================
# SPLIT TRAIN/VALIDATION
# =============================================================================

# Variables cibles
y_train = train_df[TARGET_COLS].copy()

# Split pour évaluation
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, 
    test_size=0.2, 
    random_state=42
)

print(f"Train : {X_tr.shape}")
print(f"Validation : {X_val.shape}")

---

## Étape 3 : Random Forest (Baseline)

In [ ]:
# =============================================================================
# RANDOM FOREST
# =============================================================================

print("Random Forest")
print("=" * 50)

rf_scores = {}
rf_models = {}

for target in TARGET_COLS:
    print(f"\n{target}:")
    
    # Entraîner le modèle
    rf = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        min_samples_split=5,
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_tr, y_tr[target])
    
    # Prédictions
    y_pred = rf.predict(X_val)
    
    # Scores
    r2 = r2_score(y_val[target], y_pred)
    rmse = np.sqrt(mean_squared_error(y_val[target], y_pred))
    
    rf_scores[target] = {'R2': r2, 'RMSE': rmse}
    rf_models[target] = rf
    
    print(f"  R² = {r2:.4f}")
    print(f"  RMSE = {rmse:.2f}")

# Score moyen
mean_r2_rf = np.mean([s['R2'] for s in rf_scores.values()])
print(f"\n{'=' * 50}")
print(f"R² MOYEN (Random Forest) : {mean_r2_rf:.4f}")

---

## Étape 4 : XGBoost

In [ ]:
# =============================================================================
# XGBOOST
# =============================================================================

print("XGBoost")
print("=" * 50)

xgb_scores = {}
xgb_models = {}

for target in TARGET_COLS:
    print(f"\n{target}:")
    
    # Entraîner le modèle
    xgb_model = xgb.XGBRegressor(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbosity=0
    )
    xgb_model.fit(X_tr, y_tr[target])
    
    # Prédictions
    y_pred = xgb_model.predict(X_val)
    
    # Scores
    r2 = r2_score(y_val[target], y_pred)
    rmse = np.sqrt(mean_squared_error(y_val[target], y_pred))
    
    xgb_scores[target] = {'R2': r2, 'RMSE': rmse}
    xgb_models[target] = xgb_model
    
    print(f"  R² = {r2:.4f}")
    print(f"  RMSE = {rmse:.2f}")

# Score moyen
mean_r2_xgb = np.mean([s['R2'] for s in xgb_scores.values()])
print(f"\n{'=' * 50}")
print(f"R² MOYEN (XGBoost) : {mean_r2_xgb:.4f}")

---

## Étape 5 : LightGBM

In [ ]:
# =============================================================================
# LIGHTGBM
# =============================================================================

print("LightGBM")
print("=" * 50)

lgb_scores = {}
lgb_models = {}

for target in TARGET_COLS:
    print(f"\n{target}:")
    
    # Entraîner le modèle
    lgb_model = lgb.LGBMRegressor(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbosity=-1
    )
    lgb_model.fit(X_tr, y_tr[target])
    
    # Prédictions
    y_pred = lgb_model.predict(X_val)
    
    # Scores
    r2 = r2_score(y_val[target], y_pred)
    rmse = np.sqrt(mean_squared_error(y_val[target], y_pred))
    
    lgb_scores[target] = {'R2': r2, 'RMSE': rmse}
    lgb_models[target] = lgb_model
    
    print(f"  R² = {r2:.4f}")
    print(f"  RMSE = {rmse:.2f}")

# Score moyen
mean_r2_lgb = np.mean([s['R2'] for s in lgb_scores.values()])
print(f"\n{'=' * 50}")
print(f"R² MOYEN (LightGBM) : {mean_r2_lgb:.4f}")

---

## Étape 6 : Comparaison des modèles

In [ ]:
# =============================================================================
# COMPARAISON
# =============================================================================

print("COMPARAISON DES MODÈLES")
print("=" * 60)

# Créer un tableau récapitulatif
comparison = pd.DataFrame({
    'Random Forest': [rf_scores[t]['R2'] for t in TARGET_COLS],
    'XGBoost': [xgb_scores[t]['R2'] for t in TARGET_COLS],
    'LightGBM': [lgb_scores[t]['R2'] for t in TARGET_COLS],
}, index=TARGET_NAMES)  # Utiliser les noms courts pour l'affichage

# Ajouter la moyenne
comparison.loc['MOYENNE'] = comparison.mean()

print("\nR² par variable cible :")
display(comparison.round(4))

# Meilleur modèle
best_model = comparison.loc['MOYENNE'].idxmax()
best_score = comparison.loc['MOYENNE'].max()

print(f"\n{'=' * 60}")
print(f"MEILLEUR MODÈLE : {best_model} (R² moyen = {best_score:.4f})")
print(f"\nComparaison avec baseline :")
print(f"  Avant (baseline) : R² = 0.41")
print(f"  Après (nouvelles features) : R² = {best_score:.4f}")
print(f"  Amélioration : +{(best_score - 0.41):.4f} ({((best_score - 0.41) / 0.41 * 100):.1f}%)")

In [ ]:
# =============================================================================
# VISUALISATION
# =============================================================================

fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(TARGET_COLS))
width = 0.25

bars1 = ax.bar(x - width, [rf_scores[t]['R2'] for t in TARGET_COLS], width, label='Random Forest')
bars2 = ax.bar(x, [xgb_scores[t]['R2'] for t in TARGET_COLS], width, label='XGBoost')
bars3 = ax.bar(x + width, [lgb_scores[t]['R2'] for t in TARGET_COLS], width, label='LightGBM')

# Ligne baseline
ax.axhline(y=0.41, color='red', linestyle='--', label='Baseline (0.41)')

ax.set_xlabel('Variable cible')
ax.set_ylabel('R²')
ax.set_title('Comparaison des modèles par variable cible')
ax.set_xticks(x)
ax.set_xticklabels(TARGET_NAMES)  # Noms courts
ax.legend()
ax.set_ylim(0, 1)

plt.tight_layout()
plt.show()

---

## Étape 7 : Feature Importance

In [ ]:
# =============================================================================
# FEATURE IMPORTANCE (meilleur modèle)
# =============================================================================

# Utiliser le meilleur modèle
if best_model == 'Random Forest':
    models = rf_models
elif best_model == 'XGBoost':
    models = xgb_models
else:
    models = lgb_models

fig, axes = plt.subplots(1, 3, figsize=(15, 6))

for idx, (target, target_name) in enumerate(zip(TARGET_COLS, TARGET_NAMES)):
    model = models[target]
    
    # Récupérer l'importance
    importance = model.feature_importances_
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': importance
    }).sort_values('importance', ascending=False)
    
    # Top 15
    top_features = feature_importance.head(15)
    
    ax = axes[idx]
    ax.barh(range(len(top_features)), top_features['importance'].values)
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features['feature'].values)
    ax.invert_yaxis()
    ax.set_title(f'{target_name}\n(Top 15 features)')
    ax.set_xlabel('Importance')

plt.suptitle(f'Feature Importance ({best_model})', fontsize=14)
plt.tight_layout()
plt.show()

---

## Étape 8 : Générer les prédictions pour la soumission

In [ ]:
# =============================================================================
# RÉENTRAÎNER SUR TOUTES LES DONNÉES
# =============================================================================

print(f"Réentraînement avec {best_model} sur toutes les données...")

final_models = {}

for target in TARGET_COLS:
    print(f"  {target}...", end=" ")
    
    if best_model == 'Random Forest':
        model = RandomForestRegressor(
            n_estimators=100,
            max_depth=15,
            min_samples_split=5,
            random_state=42,
            n_jobs=-1
        )
    elif best_model == 'XGBoost':
        model = xgb.XGBRegressor(
            n_estimators=200,
            max_depth=8,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=-1,
            verbosity=0
        )
    else:  # LightGBM
        model = lgb.LGBMRegressor(
            n_estimators=200,
            max_depth=8,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=-1,
            verbosity=-1
        )
    
    model.fit(X_train, y_train[target])
    final_models[target] = model
    print("OK")

print("\nModèles entraînés !")

In [ ]:
# =============================================================================
# PRÉDICTIONS SUR LE JEU DE TEST
# =============================================================================

print("Génération des prédictions...")

predictions = {}

for target in TARGET_COLS:
    pred = final_models[target].predict(X_test)
    
    # S'assurer que les prédictions sont positives
    pred = np.maximum(pred, 0)
    
    predictions[target] = pred
    print(f"  {target}: min={pred.min():.2f}, max={pred.max():.2f}, mean={pred.mean():.2f}")

print("\nPrédictions générées !")

In [ ]:
# =============================================================================
# CRÉER LE FICHIER DE SOUMISSION
# =============================================================================

# Charger le template
submission = pd.read_csv("../data/raw/submission_template.csv")

# Mapping des noms de colonnes (template -> nos noms)
# Vérifier les noms dans le template
print("Colonnes du template :")
print(list(submission.columns))

# Ajouter les prédictions avec les bons noms
for target in TARGET_COLS:
    if target in submission.columns:
        submission[target] = predictions[target]
    else:
        # Le template peut avoir des noms différents
        print(f"⚠️ Colonne '{target}' non trouvée dans le template")

# Vérifier
print(f"\nFichier de soumission :")
print(f"  Shape : {submission.shape}")

display(submission.head())

In [ ]:
# =============================================================================
# SAUVEGARDER
# =============================================================================

submission_path = "../data/submission.csv"
submission.to_csv(submission_path, index=False)

print(f"✅ Fichier de soumission sauvegardé : {submission_path}")

---

## Résumé

### Comparaison avant/après

| Métrique | Avant | Après | Amélioration |
|----------|-------|-------|-------------|
| R² moyen | 0.41 | ? | ? |
| Nb features | ~35 | ~75 | +40 |

### Nouvelles sources de données utilisées

- Landsat V2 (buffer + stats)
- TerraClimate V2 (lags + cumuls)
- ESA WorldCover (occupation du sol)
- SoilGrids (propriétés du sol)
- DEM (topographie)
- Water Type (rivière/lac)

### Fichier créé

- `submission.csv` : Prédictions pour la soumission